<a href="https://colab.research.google.com/github/mynameislllyt/API_Experiment/blob/main/Biyue_API_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
!unzip -q "/content/drive/MyDrive/API_Detection (2).zip" -d /content
%cd /content/API_Detection


/content/API_Detection


In [5]:
!python -m pip install -U pip
!grep -vE "^(torch|torchvision|torchaudio)\b" requirements.txt > requirements_notorch.txt
!pip install -r requirements_notorch.txt


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 49.7 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2


In [10]:
!grep -n "ReduceLROnPlateau" -n src/data_strengthen.py
!sed -i -E "s/, *verbose *= *[A-Za-z]+//g; s/verbose *= *[A-Za-z]+, *//g" src/data_strengthen.py
!grep -n "ReduceLROnPlateau" -n src/data_strengthen.py
!python src/data_strengthen.py --data_path data/dynamic_api_call_sequence_per_malware_100_0_306.csv --epochs 20 --batch_size 64 --lr 1e-3

292:    scheduler = optim.lr_scheduler.ReduceLROnPlateau(
292:    scheduler = optim.lr_scheduler.ReduceLROnPlateau(
使用设备: cuda

步骤 1: 加载数据
数据集统计:
  样本数量: 43876
  序列长度: 100
  API词汇表大小: 307
  正常样本数: 1079
  异常样本数: 42797

步骤 2: 数据预处理和增强

数据集划分:
  正常样本: 1079
  异常样本: 42797
滑动窗口生成:
  原始样本数: 863
  窗口大小: 10
  生成窗口数: 78533
滑动窗口生成:
  原始样本数: 216
  窗口大小: 10
  生成窗口数: 19656
滑动窗口生成:
  原始样本数: 42797
  窗口大小: 10
  生成窗口数: 3894527

最终数据集:
  训练窗口数: 78533 (全部为正常样本)
  测试窗口数: 3914183
    - 正常: 19656
    - 异常: 3894527

⚠ 注意: 正常样本数量较少 (78533 窗口)
将使用数据增强来增加训练样本...

数据增强:
  原始样本数: 78533
  增强倍数: 5
  增强后样本数: 471198 (增加了 392665 个)

步骤 3: 创建TE-LSTM模型
模型参数:
  词汇表大小: 307
  模型维度: 128
  Dropout: 0.3 (高dropout防止过拟合)
  总参数量: 566,707

步骤 4: 训练模型（使用正则化）

Epoch 1/20
--------------------------------------------------
Training: 100% 7362/7362 [01:33<00:00, 78.55it/s, loss=0.0035, l2=75.20]
Train Loss: 0.0889 | Val Loss: 0.0026
✓ 保存最佳模型 (Val Loss: 0.0026)

Epoch 2/20
--------------------------------------------------
Training: 100

In [20]:
%%writefile /content/API_Detection/src/optimize_threshold.py
"""
阈值优化脚本 - 搜索最优异常检测阈值
"""

import argparse
import os
import torch
import numpy as np
import json
from sklearn.metrics import precision_score, recall_score, f1_score
import matplotlib.pyplot as plt
from tqdm import tqdm

# 如果在Colab，添加路径
import sys
if '/content' in os.getcwd():
    sys.path.append('/content/src')

from data_preprocessing import load_api_data, prepare_train_test_data, create_dataloaders
from model import TELSTM


def find_optimal_threshold(model, test_loader, device, method='f1'):
    """
    搜索最优阈值

    核心逻辑：
    1. 计算所有测试样本的重构误差
    2. 尝试不同的阈值（从50%到99.9%分位数）
    3. 找到使F1分数最大的阈值
    """
    model.eval()
    all_errors = []
    all_labels = []

    print("计算测试集重构误差...")
    with torch.no_grad():
        for batch in tqdm(test_loader):
            x, labels = batch
            x = x.to(device)

            # 计算每个样本的重构误差
            errors = model.compute_reconstruction_error(x, reduction='mean')
            all_errors.extend(errors.cpu().numpy())
            all_labels.extend(labels.numpy())

    all_errors = np.array(all_errors)
    all_labels = np.array(all_labels)

    print(f"\n误差统计:")
    print(f"  范围: [{all_errors.min():.6f}, {all_errors.max():.6f}]")
    print(f"  均值: {all_errors.mean():.6f}")
    print(f"  中位数: {np.median(all_errors):.6f}")

    # 生成候选阈值
    percentiles = np.linspace(50, 99.9, 100)
    thresholds = np.percentile(all_errors, percentiles)

    # 搜索最优阈值
    best_score = -1
    best_threshold = None
    results = []

    print("\n搜索最优阈值...")
    for threshold in tqdm(thresholds):
        predictions = (all_errors > threshold).astype(int)

        precision = precision_score(all_labels, predictions, zero_division=0)
        recall = recall_score(all_labels, predictions, zero_division=0)
        f1 = f1_score(all_labels, predictions, zero_division=0)

        score = f1 if method == 'f1' else (precision + recall) / 2

        results.append({
            'threshold': threshold,
            'precision': precision,
            'recall': recall,
            'f1': f1,
            'score': score
        })

        if score > best_score:
            best_score = score
            best_threshold = threshold

    return best_threshold, results


def plot_threshold_curve(results, best_threshold, save_path):
    """绘制阈值-性能曲线"""
    thresholds = [r['threshold'] for r in results]
    precisions = [r['precision'] for r in results]
    recalls = [r['recall'] for r in results]
    f1s = [r['f1'] for r in results]

    plt.figure(figsize=(12, 5))

    # 全局视图
    plt.subplot(1, 2, 1)
    plt.plot(thresholds, precisions, 'b-', label='Precision', linewidth=2)
    plt.plot(thresholds, recalls, 'r-', label='Recall', linewidth=2)
    plt.plot(thresholds, f1s, 'g-', label='F1-Score', linewidth=2)
    plt.axvline(best_threshold, color='k', linestyle='--',
                label=f'Best={best_threshold:.6f}', linewidth=2)
    plt.xlabel('Threshold')
    plt.ylabel('Score')
    plt.title('Metrics vs Threshold')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.xscale('log')

    # 局部放大
    plt.subplot(1, 2, 2)
    idx = np.argmin(np.abs(np.array(thresholds) - best_threshold))
    start, end = max(0, idx-20), min(len(thresholds), idx+20)

    plt.plot(thresholds[start:end], precisions[start:end], 'b-', label='Precision', linewidth=2)
    plt.plot(thresholds[start:end], recalls[start:end], 'r-', label='Recall', linewidth=2)
    plt.plot(thresholds[start:end], f1s[start:end], 'g-', label='F1-Score', linewidth=2)
    plt.axvline(best_threshold, color='k', linestyle='--', linewidth=2)
    plt.xlabel('Threshold')
    plt.ylabel('Score')
    plt.title('Zoomed View')
    plt.legend()
    plt.grid(True, alpha=0.3)

    plt.tight_layout()
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    print(f"阈值曲线已保存: {save_path}")


def main(args):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"使用设备: {device}")

    # 加载模型
    print("\n加载模型...")
    checkpoint = torch.load(args.model_path, map_location=device)
    model_args = checkpoint['args']
    vocab_size = checkpoint['vocab_size']

    model = TELSTM(
        vocab_size=vocab_size,
        d_model=model_args['d_model'],
        nhead=model_args['nhead'],
        num_encoder_layers=model_args['num_layers'],
        lstm_hidden=model_args['lstm_hidden'],
        dropout=model_args['dropout']
    ).to(device)

    model.load_state_dict(checkpoint['model_state_dict'])
    model.eval()

    # 加载数据
    print("\n加载数据...")
    df, _ = load_api_data(args.data_path)
    train_windows, test_windows, test_labels = prepare_train_test_data(
        df,
        window_size=model_args['window_size'],
        test_ratio=model_args['test_ratio'],
        random_state=model_args['seed']
    )

    _, test_loader = create_dataloaders(
        train_windows, test_windows, test_labels,
        batch_size=args.batch_size
    )

    # 搜索最优阈值
    print("\n" + "="*50)
    print("搜索最优阈值")
    print("="*50)

    best_threshold, results = find_optimal_threshold(
        model, test_loader, device, method=args.method
    )

    print(f"\n✓ 最优阈值: {best_threshold:.6f}")

    # 保存结果
    os.makedirs(args.result_dir, exist_ok=True)

    threshold_info = {
        'optimal_threshold': float(best_threshold),
        'method': args.method
    }

    threshold_path = os.path.join(args.result_dir, 'optimal_threshold.json')
    with open(threshold_path, 'w') as f:
        json.dump(threshold_info, f, indent=4)
    print(f"阈值已保存: {threshold_path}")

    # 更新模型目录的阈值
    model_dir = os.path.dirname(args.model_path)
    model_threshold_path = os.path.join(model_dir, 'threshold.json')
    with open(model_threshold_path, 'w') as f:
        json.dump({'threshold': float(best_threshold), 'percentile': None}, f, indent=4)
    print(f"模型阈值已更新: {model_threshold_path}")

    # 绘制曲线
    plot_path = os.path.join(args.result_dir, 'threshold_analysis.png')
    plot_threshold_curve(results, best_threshold, plot_path)

    print("\n✓ 优化完成！")


if __name__ == "__main__":
    parser = argparse.ArgumentParser(description='优化异常检测阈值')

    parser.add_argument('--model_path', type=str, required=True,
                       help='模型文件路径')
    parser.add_argument('--data_path', type=str, required=True,
                       help='数据集路径')
    parser.add_argument('--batch_size', type=int, default=32,
                       help='批次大小')
    parser.add_argument('--method', type=str, default='f1',
                       choices=['f1', 'balanced'],
                       help='优化目标')
    parser.add_argument('--result_dir', type=str, default='results',
                       help='结果保存目录')

    args = parser.parse_args()
    main(args)



Overwriting /content/API_Detection/src/optimize_threshold.py


In [21]:
# 方式2：使用命令行
!python /content/API_Detection/src/optimize_threshold.py \
    --model_path /content/API_Detection/models/te_lstm_best_augmented.pth \
    --data_path /content/API_Detection/data/dynamic_api_call_sequence_per_malware_100_0_306.csv


使用设备: cuda

加载模型...

加载数据...
数据集统计:
  样本数量: 43876
  序列长度: 100
  API词汇表大小: 307
  正常样本数: 1079
  异常样本数: 42797

数据集划分:
  正常样本: 1079
  异常样本: 42797
滑动窗口生成:
  原始样本数: 863
  窗口大小: 10
  生成窗口数: 78533
滑动窗口生成:
  原始样本数: 216
  窗口大小: 10
  生成窗口数: 19656
滑动窗口生成:
  原始样本数: 42797
  窗口大小: 10
  生成窗口数: 3894527

最终数据集:
  训练窗口数: 78533 (全部为正常样本)
  测试窗口数: 3914183
    - 正常: 19656
    - 异常: 3894527

搜索最优阈值
计算测试集重构误差...
100% 122319/122319 [04:08<00:00, 491.91it/s]

误差统计:
  范围: [0.000011, 9.994293]
  均值: 0.022278
  中位数: 0.000196

搜索最优阈值...
100% 100/100 [00:50<00:00,  1.97it/s]

✓ 最优阈值: 0.000196
阈值已保存: results/optimal_threshold.json
模型阈值已更新: /content/API_Detection/models/threshold.json
阈值曲线已保存: results/threshold_analysis.png

✓ 优化完成！


In [23]:
%%writefile src/evaluate.py
"""
模型评估脚本 - 完整的性能评估
"""

import argparse
import os
import json
import torch
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    confusion_matrix, classification_report, roc_curve, auc
)
from tqdm import tqdm

import sys
if '/content' in os.getcwd():
    sys.path.append('/content/src')

from data_preprocessing import load_api_data, prepare_train_test_data, create_dataloaders
from model import TELSTM


def evaluate_model(model, test_loader, threshold, device):
    """评估模型性能"""
    model.eval()

    all_predictions = []
    all_labels = []
    all_errors = []

    print("评估中...")
    with torch.no_grad():
        for batch in tqdm(test_loader):
            x, labels = batch
            x = x.to(device)

            # 计算重构误差
            errors = model.compute_reconstruction_error(x, reduction='mean')

            # 基于阈值判断异常
            predictions = (errors > threshold).long()

            all_predictions.extend(predictions.cpu().numpy())
            all_labels.extend(labels.numpy())
            all_errors.extend(errors.cpu().numpy())

    return np.array(all_predictions), np.array(all_labels), np.array(all_errors)


def plot_confusion_matrix(cm, save_path):
    """绘制混淆矩阵"""
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=['Normal', 'Anomaly'],
                yticklabels=['Normal', 'Anomaly'])
    plt.title('Confusion Matrix', fontsize=14, fontweight='bold')
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.tight_layout()
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    print(f"混淆矩阵已保存: {save_path}")


def plot_roc_curve(labels, errors, save_path):
    """绘制ROC曲线"""
    fpr, tpr, _ = roc_curve(labels, errors)
    roc_auc = auc(fpr, tpr)

    plt.figure(figsize=(8, 6))
    plt.plot(fpr, tpr, color='darkorange', lw=2,
             label=f'ROC curve (AUC = {roc_auc:.4f})')
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curve', fontsize=14, fontweight='bold')
    plt.legend(loc="lower right")
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    print(f"ROC曲线已保存: {save_path}")

    return roc_auc


def plot_error_distribution(labels, errors, threshold, save_path):
    """绘制误差分布"""
    normal_errors = errors[labels == 0]
    anomaly_errors = errors[labels == 1]

    plt.figure(figsize=(10, 6))
    plt.hist(normal_errors, bins=50, alpha=0.6, label='Normal',
             color='green', density=True)
    plt.hist(anomaly_errors, bins=50, alpha=0.6, label='Anomaly',
             color='red', density=True)
    plt.axvline(threshold, color='black', linestyle='--', linewidth=2,
                label=f'Threshold={threshold:.6f}')
    plt.xlabel('Reconstruction Error')
    plt.ylabel('Density')
    plt.title('Error Distribution', fontsize=14, fontweight='bold')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    print(f"误差分布已保存: {save_path}")


def main(args):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"使用设备: {device}")

    # 加载模型
    print("\n" + "="*50)
    print("加载模型")
    print("="*50)

    checkpoint = torch.load(args.model_path, map_location=device)
    model_args = checkpoint['args']
    vocab_size = checkpoint['vocab_size']

    model = TELSTM(
        vocab_size=vocab_size,
        d_model=model_args['d_model'],
        nhead=model_args['nhead'],
        num_encoder_layers=model_args['num_layers'],
        lstm_hidden=model_args['lstm_hidden'],
        dropout=model_args['dropout']
    ).to(device)

    model.load_state_dict(checkpoint['model_state_dict'])
    print(f"✓ 模型已加载: {args.model_path}")

    # 加载阈值
    model_dir = os.path.dirname(args.model_path)
    threshold_path = os.path.join(model_dir, 'threshold.json')

    if os.path.exists(threshold_path):
        with open(threshold_path, 'r') as f:
            threshold_info = json.load(f)
            threshold = threshold_info['threshold']
        print(f"✓ 使用阈值: {threshold:.6f}")
    else:
        print("⚠ 未找到阈值文件，使用默认值 0.0004")
        threshold = 0.0004

    # 加载数据
    print("\n" + "="*50)
    print("加载数据")
    print("="*50)

    df, _ = load_api_data(args.data_path)
    train_windows, test_windows, test_labels = prepare_train_test_data(
        df,
        window_size=model_args['window_size'],
        test_ratio=model_args['test_ratio'],
        random_state=model_args['seed']
    )

    _, test_loader = create_dataloaders(
        train_windows, test_windows, test_labels,
        batch_size=args.batch_size
    )

    # 评估
    print("\n" + "="*50)
    print("评估模型")
    print("="*50)

    predictions, labels, errors = evaluate_model(
        model, test_loader, threshold, device
    )

    # 计算指标
    accuracy = accuracy_score(labels, predictions)
    precision = precision_score(labels, predictions, zero_division=0)
    recall = recall_score(labels, predictions, zero_division=0)
    f1 = f1_score(labels, predictions, zero_division=0)
    cm = confusion_matrix(labels, predictions)

    fpr, tpr, _ = roc_curve(labels, errors)
    roc_auc = auc(fpr, tpr)

    # 打印结果
    print("\n" + "="*50)
    print("评估结果")
    print("="*50)
    print(f"\n准确率 (Accuracy):  {accuracy:.4f}")
    print(f"精确率 (Precision): {precision:.4f}")
    print(f"召回率 (Recall):    {recall:.4f}")
    print(f"F1分数 (F1-Score):  {f1:.4f}")
    print(f"AUC-ROC:            {roc_auc:.4f}")

    print(f"\n混淆矩阵:")
    print(f"                预测正常  预测异常")
    print(f"实际正常    {cm[0, 0]:>8}  {cm[0, 1]:>8}")
    print(f"实际异常    {cm[1, 0]:>8}  {cm[1, 1]:>8}")

    # 误差统计
    normal_errors = errors[labels == 0]
    anomaly_errors = errors[labels == 1]

    print(f"\n误差统计:")
    print(f"正常样本: {normal_errors.mean():.4f} (±{normal_errors.std():.4f})")
    print(f"异常样本: {anomaly_errors.mean():.4f} (±{anomaly_errors.std():.4f})")

    # 保存结果
    os.makedirs(args.result_dir, exist_ok=True)

    # 保存指标
    metrics = {
        'accuracy': float(accuracy),
        'precision': float(precision),
        'recall': float(recall),
        'f1_score': float(f1),
        'auc': float(roc_auc),
        'threshold': float(threshold),
        'confusion_matrix': cm.tolist()
    }

    metrics_path = os.path.join(args.result_dir, 'evaluation_metrics.json')
    with open(metrics_path, 'w') as f:
        json.dump(metrics, f, indent=4)
    print(f"\n✓ 指标已保存: {metrics_path}")

    # 保存详细报告
    report = classification_report(labels, predictions,
                                   target_names=['Normal', 'Anomaly'])
    report_path = os.path.join(args.result_dir, 'classification_report.txt')
    with open(report_path, 'w') as f:
        f.write(report)
    print(f"✓ 详细报告已保存: {report_path}")

    # 绘制图表
    print("\n生成可视化...")
    plot_confusion_matrix(cm, os.path.join(args.result_dir, 'confusion_matrix.png'))
    plot_roc_curve(labels, errors, os.path.join(args.result_dir, 'roc_curve.png'))
    plot_error_distribution(labels, errors, threshold,
                           os.path.join(args.result_dir, 'error_distribution.png'))

    print("\n" + "="*50)
    print("评估完成！")
    print("="*50)
    print(f"结果保存于: {args.result_dir}")


if __name__ == "__main__":
    parser = argparse.ArgumentParser(description='评估TE-LSTM模型')

    parser.add_argument('--model_path', type=str, required=True,
                       help='模型文件路径')
    parser.add_argument('--data_path', type=str, required=True,
                       help='数据集路径')
    parser.add_argument('--batch_size', type=int, default=32,
                       help='批次大小')
    parser.add_argument('--result_dir', type=str, default='results',
                       help='结果保存目录')

    args = parser.parse_args()
    main(args)


Overwriting src/evaluate.py


In [24]:
!python src/evaluate.py --model_path models/te_lstm_best_augmented.pth --data_path data/dynamic_api_call_sequence_per_malware_100_0_306.csv


使用设备: cuda

加载模型
✓ 模型已加载: models/te_lstm_best_augmented.pth
✓ 使用阈值: 0.000196

加载数据
数据集统计:
  样本数量: 43876
  序列长度: 100
  API词汇表大小: 307
  正常样本数: 1079
  异常样本数: 42797

数据集划分:
  正常样本: 1079
  异常样本: 42797
滑动窗口生成:
  原始样本数: 863
  窗口大小: 10
  生成窗口数: 78533
滑动窗口生成:
  原始样本数: 216
  窗口大小: 10
  生成窗口数: 19656
滑动窗口生成:
  原始样本数: 42797
  窗口大小: 10
  生成窗口数: 3894527

最终数据集:
  训练窗口数: 78533 (全部为正常样本)
  测试窗口数: 3914183
    - 正常: 19656
    - 异常: 3894527

评估模型
评估中...
100% 122319/122319 [04:17<00:00, 474.37it/s]

评估结果

准确率 (Accuracy):  0.4993
精确率 (Precision): 0.9961
召回率 (Recall):    0.4987
F1分数 (F1-Score):  0.6646
AUC-ROC:            0.5926

混淆矩阵:
                预测正常  预测异常
实际正常       12115      7541
实际异常     1952479   1942048

误差统计:
正常样本: 0.0028 (±0.0426)
异常样本: 0.0224 (±0.2052)

✓ 指标已保存: results/evaluation_metrics.json
✓ 详细报告已保存: results/classification_report.txt

生成可视化...
混淆矩阵已保存: results/confusion_matrix.png
ROC曲线已保存: results/roc_curve.png
误差分布已保存: results/error_distribution.png

评估完成！
结果保存于: results
